In [1]:
from Bio import SeqIO
import pandas as pd
from BCBio import GFF


In [3]:
%cd "/home/andrei/Data/HeteroR/test_dir/GRF"
rgi = pd.read_csv("DA62886_rgi_table.tsv", sep="\t")

/home/andrei/Data/HeteroR/test_dir/GRF


In [5]:
rgi.columns

Index(['ORF_ID', 'Contig', 'Start', 'Stop', 'Orientation', 'Cut_Off',
       'Pass_Bitscore', 'Best_Hit_Bitscore', 'Best_Hit_ARO', 'Best_Identities',
       'ARO', 'Model_type', 'SNPs_in_Best_Hit_ARO', 'Other_SNPs', 'Drug Class',
       'Resistance Mechanism', 'AMR Gene Family', 'Predicted_DNA',
       'Predicted_Protein', 'CARD_Protein_Sequence',
       'Percentage Length of Reference Sequence', 'ID', 'Model_ID', 'Nudged',
       'Note'],
      dtype='object')

In [6]:
rgi["Cut_Off"].unique()

array(['Loose', 'Strict', 'Perfect'], dtype=object)

In [15]:
pd.set_option('display.max_columns', None)
rgi_notLoose = rgi[rgi["Cut_Off"] != "Loose"]
rgi_notLoose

,ORF_ID,Contig,Start,Stop,Orientation,Cut_Off,Pass_Bitscore,Best_Hit_Bitscore,Best_Hit_ARO,Best_Identities,ARO,Model_type,SNPs_in_Best_Hit_ARO,Other_SNPs,Drug Class,Resistance Mechanism,AMR Gene Family,Predicted_DNA,Predicted_Protein,CARD_Protein_Sequence,Percentage Length of Reference Sequence,ID,Model_ID,Nudged,Note
93,HOKEGFAO_00108 Multidrug transporter EmrE,NaN,NaN,NaN,NaN,Strict,190,216.083,Escherichia coli emrE,98.18,3004039,protein homolog model,NaN,NaN,macrolide antibiotic,antibiotic efflux,small multidrug resistance (SMR) antibiotic ef...,NaN,MNPYIYLGGAILAEVIGTTLMKFSEGFTRLWPSVGTIICYCASFWL...,MNPYIYLGGAILAEVIGTTLMKFSEGFTRLWPSVGTIICYCASFWL...,100.00,gnl|BL_ORD_ID|2380|hsp_num:0,2656,NaN,NaN
301,HOKEGFAO_00337 UDP-glucose 6-dehydrogenase,NaN,NaN,NaN,NaN,Strict,700,759.599,ugd,94.59,3003577,protein homolog model,NaN,NaN,peptide antibiotic,antibiotic target alteration,pmr phosphoethanolamine transferase,NaN,MKITISGTGYVGLSNGLLISQNHNVVALDILPSRVAMLNDRISPIV...,MKITISGTGYVGLSNGLLIAQNHEVVALDILPSRVAMLNDRISPIV...,100.00,gnl|BL_ORD_ID|916|hsp_num:0,842,NaN,NaN
347,HOKEGFAO_00388 Multidrug resistance protein MdtA,NaN,NaN,NaN,NaN,Strict,725,826.239,mdtA,98.80,3000792,protein homolog model,NaN,NaN,aminocoumarin antibiotic,antibiotic efflux,resistance-nodulation-cell division (RND) anti...,NaN,MKGSYKSRWVIVIVVVIAAIAAFWFWQGRNDSQSAAPGATKQAQQS...,MKGSYKSRWVIVIVVVIAAIAAFWFWQGRNDSRSAAPGATKQAQQS...,100.00,gnl|BL_ORD_ID|360|hsp_num:0,387,NaN,NaN
348,HOKEGFAO_00389 Multidrug resistance protein MdtB,NaN,NaN,NaN,NaN,Strict,1800,2076.600,mdtB,99.71,3000793,protein homolog model,NaN,NaN,aminocoumarin antibiotic,antibiotic efflux,resistance-nodulation-cell division (RND) anti...,NaN,MQVLPPSSTGGPSRLFIMRPVATTLLMVAILLAGIIGYRALPVSAL...,MQVLPPSSTGGPSRLFIMRPVATTLLMVAILLAGIIGYRALPVSAL...,100.00,gnl|BL_ORD_ID|776|hsp_num:0,820,NaN,NaN
349,HOKEGFAO_00390 Multidrug resistance protein MdtC,NaN,NaN,NaN,NaN,Strict,1800,2061.960,mdtC,99.51,3000794,protein homolog model,NaN,NaN,aminocoumarin antibiotic,antibiotic efflux,resistance-nodulation-cell division (RND) anti...,NaN,MKFFALFIYRPVATILLSVAITLCGILGFRMLPVAPLPQVDFPVIM...,MKFFALFIYRPVATILLSVAITLCGILGFRMLPVAPLPQVDFPVII...,100.00,gnl|BL_ORD_ID|1241|hsp_num:0,1315,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,HOKEGFAO_04953 Dihydropteroate synthase,NaN,NaN,NaN,NaN,Perfect,500,528.865,sul2,100.00,3000412,protein homolog model,NaN,NaN,sulfonamide antibiotic,antibiotic target replacement,sulfonamide resistant sul,NaN,MNKSLIIFGIVNITSDSFSDGGRYLAPDAAIAQARKLMAEGADVID...,MNKSLIIFGIVNITSDSFSDGGRYLAPDAAIAQARKLMAEGADVID...,100.00,gnl|BL_ORD_ID|894|hsp_num:0,952,NaN,NaN
4425,HOKEGFAO_04954 hypothetical protein,NaN,NaN,NaN,NaN,Strict,500,355.140,APH(3'')-Ib,99.43,3002639,protein homolog model,NaN,NaN,aminoglycoside antibiotic,antibiotic inactivation,APH(3''),NaN,MNRTNIFFGESHSDWLPVRGGESGDFVFRRGDGHAFAKIAPASRRG...,MNRTNIFFGESHSDWLPVRGGESGDFVFRRGDGHAFAKIAPASRRG...,67.42,gnl|BL_ORD_ID|1729|hsp_num:0,1830,True,loose hit with at least 95 percent identity pu...
4426,HOKEGFAO_04955 Dihydrofolate reductase type 1,NaN,NaN,NaN,NaN,Strict,300,327.791,dfrA14,98.73,3002859,protein homolog model,NaN,NaN,diaminopyrimidine antibiotic,antibiotic target replacement,trimethoprim resistant dihydrofolate reductase...,NaN,MRTLKVSLIAAKAKNGVIGCGPDIPWSAKGEQLLFKALTYNQWLLV...,MKVSLMAAKAKNGVIGCGPDIPWSAKGEQLLFKALTYNQWLLVGRK...,101.91,gnl|BL_ORD_ID|1496|hsp_num:0,1592,NaN,NaN
4427,HOKEGFAO_04956 Aminoglycoside 3'-phosphotransf...,NaN,NaN,NaN,NaN,Strict,500,166.007,APH(3'')-Ib,100.00,3002639,protein homolog model,NaN,NaN,aminoglycoside antibiotic,antibiotic inactivation,APH(3''),NaN,MPNFMVDPKTLQCTGLIDLGRLGTADRYADLALMIANAEENWAAPD...,MNRTNIFFGESHSDWLPVRGGESGDFVFRRGDGHAFAKIAPASRRG...,30.34,gnl|BL_ORD_ID|1729|hsp_num:0,1830,True,loose hit with at least 95 percent identity pu...


In [16]:
in_file = "DA62886_genomic.gff"

with open(in_file) as f:
    gff = [rec for rec in GFF.parse(f)]

chromosome_genes = [feature for feature in gff[0].features if feature.type == "gene"]  # here we have IDs and positions

# filter the genes with resistance - both in RGI and GBK
# naive looping approach
resistance_genes_coords = list()
for orf in rgi_notLoose["ORF_ID"]:
    for gene in chromosome_genes:
        if orf.split(" ")[0] in gene.id:
            resistance_genes_coords.append(gene)


In [20]:
# less naive approach
# make sets

rgi_set = set([item.split(" ")[0] for item in rgi_notLoose["ORF_ID"]])

chrom_genes_set = set([item.id[:-5] for item in chromosome_genes])

In [24]:
len(rgi_set)

61

In [25]:
len(chrom_genes_set)

4947

In [27]:
len(chrom_genes_set.intersection(rgi_set))

55

Annotation file lacks some RG? (6)

In [29]:
rg_found = chrom_genes_set.intersection(rgi_set)
rg_notFound = rgi_set.difference(rg_found)
rg_notFound


{'HOKEGFAO_04949',
 'HOKEGFAO_04953',
 'HOKEGFAO_04954',
 'HOKEGFAO_04955',
 'HOKEGFAO_04956',
 'HOKEGFAO_04957'}